# text.data

> Fill in a module description here


In [ ]:
#| default_exp text.data

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import copy
import random
import json
from dataclasses import dataclass
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset
from torch.utils.data import random_split
import transformers
from fastcore.utils import *

In [ ]:
path = '/home/ahmed/Ahmed-home/1- Projects/Research/publications/2024/letter 1/code/PFLlib/dataset/Cifar10/'

In [ ]:
DATA_DIR = '/home/ahmed/Ahmed-home/1- Projects/Research/publications/2024/letter 1/code/mira/data/'

In [ ]:
def load_json(file_path):
    # Determine if the file is JSON or JSONL
    is_jsonl = file_path.endswith('.jsonl')

    # Read the data from the file
    if is_jsonl:
        with open(file_path, 'r') as file:
            input_data = [json.loads(line) for line in file]
    else:
        with open(file_path, 'r') as file:
            input_data = json.load(file)

    return input_data, is_jsonl

In [ ]:

def transform_json(input_data, is_jsonl, schema):
    """
    Transforms a JSON or JSONL file based on the provided schema.

    Args:
        file_path (str): Path to the JSON or JSONL file.
        schema (dict): A dictionary defining the mapping of keys from the input JSON to the output data.

    Returns:
        list: A list of transformed dictionaries based on the schema.
    """
    
    # The transformed data to return
    transformed_data = []

    # Process JSONL (list of records) or JSON with a main key
    if is_jsonl:
        main_data = input_data
    else:
        main_key = next(iter(schema.values()))
        if not isinstance(main_key, dict):
            raise ValueError("Schema must specify a dictionary mapping for the main data key.")

        primary_key = list(main_key.keys())[0]
        main_data = input_data.get(primary_key, [])
        if not isinstance(main_data, list):
            raise ValueError(f"Expected a list of objects under key '{primary_key}', got {type(main_data)}.")

    # Transform each object in the main data list
    for obj in main_data:
        transformed_obj = {}
        for output_key, mapping in schema.items():
            if mapping is None:
                transformed_obj[output_key] = ""
            elif isinstance(mapping, dict):
                inner_key, source_key = next(iter(mapping.items()))
                nested_data = obj.get(source_key, None)
                # Handle nested list values, taking only the first item
                if isinstance(nested_data, list):
                    nested_data = nested_data[0] if nested_data else None
                transformed_obj[output_key] = nested_data
            elif isinstance(mapping, str) and not is_jsonl:
                transformed_obj[output_key] = input_data.get(mapping, None)[0]
            else:
                value = obj.get(mapping, None)
                # Handle list values, taking only the first item
                if isinstance(value, list):
                    value = value[0] if value else None
                transformed_obj[output_key] = value
        transformed_data.append(transformed_obj)

    return transformed_data



In [ ]:
def _get_task_splits(dir_name, extenstion):
    with open(os.path.join(DATA_DIR, dir_name, 'splits', 'default', 'train_tasks.txt'), 'r') as reader:
        task_names = [f'{content.strip()}.{extenstion}' for content in reader.readlines()]
        
    return task_names

In [ ]:
def read_llm_data_from_dir(dir_name,
                           tasks_splitted= False,
                           extenstion= 'json',
                           schema= None,
                           preprocess_fn = None,
                           **kwargs):
    if tasks_splitted:
        task_names = _get_task_splits(dir_name, extenstion= extenstion)
    
    else:
        task_names = [file for file in os.listdir(os.path.join(DATA_DIR, dir_name, "tasks")) if file.endswith(f'.{extenstion}')]

    data = []
    for idx, file_name in enumerate(task_names):
        input_data, is_jsonl = load_json(os.path.join(DATA_DIR, dir_name,"tasks", file_name))
        transformed_data = transform_json(input_data, is_jsonl, schema)
        data.extend(transformed_data)

    if preprocess_fn:
        data = preprocess_fn(data, **kwargs)

    return data

In [ ]:
# dir_name = "natural-instructions-2.8"

# schema = {
#     "input": {"Instances": "input"},
#     "context": None,
#     "ouput": {"Instances": "output"},
#     "task": "Categories",
#     "instruction": "Definition",
# }

# preprocess_fn = lambda items, max_length: [item for item in items if len(item['input']) < max_length]  # noqa: E731


# # write the above function with lambda
# data = read_llm_data_from_dir(dir_name,
#                               tasks_splitted= True,
#                               schema= schema,
#                               preprocess_fn= preprocess_fn,
#                               max_length= 512)


In [ ]:
# from sklearn.model_selection import train_test_split
# train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# import pandas as pd
# df = pd.DataFrame(train_set)

In [ ]:
# df

,input,context,ouput,task,instruction
0,I wish these would have worked. I bought them ...,,2,Sentiment Analysis,"In this task, you're given a review from Amazo..."
1,emotion is part of mind,,emotion PartOf mind,Information Extraction,Given a phrase describing the relationship bet...
2,"February 20, 1981 doesn't occur between 03 Jul...",,False,Misc.,"Given a statement about date and time, state w..."
3,Problem: there are 6 red balls and 4 blue ball...,,b,Question Answering,"In this task, you need to provide the correct ..."
4,purpose is a kind of goal,,purpose IsA goal,Information Extraction,Given a phrase describing the relationship bet...
...,...,...,...,...,...
1632953,As hard as I tried I just couldn't smell that ...,,Negative,Sentiment Analysis,You are given a review of Amazon's food produc...
1632954,Beginning: Charlene wanted to make a spaghetti...,,Charlene's kids ate a can of cake frosting for...,Story Composition,"In this task, you're given the beginning and t..."
1632955,Question: Is a new polymorphism in the GRP78 a...,,no,Answer Verification,Given a passage with a question and an answer ...
1632956,Question: Jane was looking at different object...,,it would decrease,Question Answering,You are given a question or fill-in-the-blank ...


In [ ]:
# data

[{'input': "He points at the mohawked camper's brightly painted van. The henchman",
  'context': '',
  'ouput': 'flies up from his seat and rubs its eye.',
  'task': 'Wrong Candidate Generation',
  'instruction': 'Given a sentence, generate a most unlikely next statement. The next statement should not be logically relevant to the given context'},
 {'input': "Fact1: Desmond 's mother-in-law reports situation to Irish authorities, Fact2: children are prohibited by law to being left in broken homes, Fact3: Evelyn 's grandfather takes her to girls orphanage, Fact4: little hope in regaining custody of children because Desmond can not afford lawyer turns to drink, Fact5: Bernadette tells Desmond to go for help at Michael 's",
  'context': '',
  'ouput': 'Nineyearold Evelyn Doyle Sophie Vavasseur and her two brothers, Maurice Hugh MacDonagh and Dermot Niall Beagan are motherless when their mother abandons the marriage to a drunkard outofwork father Desmond Doyle Pierce Brosnan. When Desmonds 

In [ ]:
# file_path = os.path.join('data', 'databricks-dolly-15k.jsonl')
# key_dsc1 = {'instruction': 'instruction', 
#            'context': 'context',
#            "output": "response",
#            "task": "category"}

# data = transform_json(file_path, schema= key_dsc1)

In [ ]:
# data

[{'instruction': 'When did Virgin Australia start operating?',
  'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.",
  'output': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.',
  'task': 'closed_qa'},
 {'instruction': 'Which is a species of fish? Tope or Rope',
  'context': '',
  'output': 'Tope',
  'task': 'classification'},
 {'instruction': 'Why can camels survive for long without water?',
  'context': '',
  'output': 'Camels use the fat in their humps to kee

In [ ]:
def _filter_out_over_length(items, max_length):
    return [item for item in items if len(item['input']) < max_length]


In [ ]:
# from datasets import load_dataset

# ds = load_dataset("fancyzhx/ag_news")

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
# ds['train'].to_pandas()

,text,label
0,Wall St. Bears Claw Back Into the Black (Reute...,2
1,Carlyle Looks Toward Commercial Aerospace (Reu...,2
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2
3,Iraq Halts Oil Exports from Main Southern Pipe...,2
4,"Oil prices soar to all-time record, posing new...",2
...,...,...
119995,Pakistan's Musharraf Says Won't Quit as Army C...,0
119996,Renteria signing a top-shelf deal Red Sox gene...,1
119997,Saban not going to Dolphins yet The Miami Dolp...,1
119998,Today's NFL games PITTSBURGH at NY GIANTS Time...,1


In [ ]:
#| export
from enum import Enum
class DefaultToken(Enum):
    PAD_TOKEN = "[PAD]"
    EOS_TOKEN = "</s>"
    BOS_TOKEN = "<s>"
    UNK_TOKEN = "<unk>"
    IGNORE_INDEX = -100


In [ ]:
#| export
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, "
        "paired with an input that provides further context. "
        "Write a response for the task request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:"
        "\n{input}\n\n### Response:"),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response for the task request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"),
}


In [ ]:

@dataclass
class LLMDataCollator(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances):
        input_ids, labels = tuple([instance[key] for instance in instances]
                                  for key in ("input_ids", "labels"))
        
        input_ids = torch.nn.utils.rnn.pad_sequence(
                                                    input_ids,
                                                    batch_first=True,
                                                    padding_value=self.tokenizer.pad_token_id)
        labels = torch.nn.utils.rnn.pad_sequence(
                                                labels,
                                                batch_first=True,
                                                padding_value=DefaultToken.IGNORE_INDEX.value)
        
        categories = torch.tensor([instance["categories"] for instance in instances])

        return dict(
            input_ids=input_ids,
            labels=labels,
            categories=categories,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )


In [ ]:
#| export 
class MTLDataSet(Dataset):
    def __init__(self,
                 list_data_dict,
                 tokenizer,
                 prompt_input=PROMPT_DICT["prompt_input"],
                 prompt_no_input=PROMPT_DICT["prompt_no_input"], 
                 generation=False):
        """
            list_data_dict: list of dictionaries with keys 'input', 'output', 'category'
        """
        
        super(MTLDataSet, self).__init__()
            
        sources = [
            prompt_input.format_map(example) if example.get("input", "") != ""
            else prompt_no_input.format_map(example)
            for example in list_data_dict
        ]
        targets = [
            f"{example['output']}{tokenizer.eos_token}"
            for example in list_data_dict
        ]

        data_dict = self.preprocess(sources, targets, tokenizer, generation=generation)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

        categories = [
            example['category'] if 'category' in example else None
            for example in list_data_dict
        ]
        self.tasks = categories
        df = pd.DataFrame(categories, columns=["category"])
        self.categories = list(pd.Categorical(df["category"]).codes)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i):
        return dict(input_ids=self.input_ids[i],
                    labels=self.labels[i],
                    categories=self.categories[i],
                    tasks=self.tasks[i])

In [ ]:
#| export 
@patch
def _tokenize_fn(self: MTLDataSet, strings, tokenizer):
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                max_length=tokenizer.model_max_length,
                truncation=True,
                padding=True,
            ) for text in strings
        ]
        input_ids = labels = [
            tokenized.input_ids[0] for tokenized in tokenized_list
        ]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item()
            for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )

In [ ]:
#| export
@patch
def preprocess(self: MTLDataSet, sources, targets, tokenizer, generation):
        if generation:
            sources_tokenized, labels_tokenized = [
                self._tokenize_fn(strings, tokenizer)
                for strings in (sources, targets)
            ]
            input_ids = self._tokenize_fn(sources, tokenizer)["input_ids"]
            labels = self._tokenize_fn(targets, tokenizer)["input_ids"]
        else:
            examples = [s + t for s, t in zip(sources, targets)]
            examples_tokenized, sources_tokenized = [
                self._tokenize_fn(strings, tokenizer)
                for strings in (examples, sources)
            ]
            input_ids = examples_tokenized["input_ids"]
            labels = copy.deepcopy(input_ids)
            for label, source_len in zip(labels,
                                        sources_tokenized["input_ids_lens"]):
                label[:source_len] = DefaultToken.IGNORE_INDEX.value
        return dict(input_ids=input_ids, labels=labels)

In [ ]:
# #| export 

# def split_by_category(list_data_dict):
#         groupdict = dict()
#         for example in list_data_dict:
#             if example['category'] not in groupdict:
#                 groupdict[example['category']] = []
#             groupdict[example['category']].append(example)
#         return groupdict


# def train_eval_split(dataset):
    
#     dataset_size = len(dataset)
#     train_size = int(0.8 * dataset_size)  # 80% for training
#     val_size = dataset_size - train_size  # Remaining 20% for validation

#     train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

#     return train_dataset, val_dataset


# def split_tasks_among_clients(tasks_dict, num_clients=10):
#     result = {task: [[] for _ in range(num_clients)] for task in tasks_dict}
    
#     for task, samples in tasks_dict.items():
#         # Shuffle the samples to ensure randomness
#         shuffled_samples = samples.copy()
#         random.shuffle(shuffled_samples)
        
#         # Distribute samples among clients
#         for i, sample in enumerate(shuffled_samples):
#             client_index = random.randint(0, num_clients - 1)
#             result[task][client_index].append(sample)
    
#     return result

# def get_dolly(args, tokenizer):
#     data_collator = LLMDataCollator(tokenizer=tokenizer)
#     json_name = 'databricks-dolly-15k.jsonl'
#     list_data_dict =  load_jsonl(os.path.join('data', json_name), 
#                             instruction='instruction',
#                             input='context',
#                             output='response',
#                             category='category')
    
#     grouped_data = split_by_category(list_data_dict)
#     result = split_tasks_among_clients(grouped_data)

#     lst_train_ds = []
#     lst_eval_set = []
#     lst_train_ds_genr = []
#     lst_eval_set_genr = []

#     for task in result:
#         for client_data in result[task]:
#             dataset = MTLDataSet(client_data, tokenizer, generation=False)
#             train_loader, val_loader = train_eval_split(dataset)
#             lst_train_ds.append(train_loader)
#             lst_eval_set.append(val_loader)

#             dataset_genr = MTLDataSet(client_data, tokenizer, generation=True)
#             train_ds_genr, val_loader_genr = train_eval_split(dataset_genr)
#             lst_train_ds_genr.append(train_ds_genr)
#             lst_eval_set_genr.append(val_loader_genr)

#     return (lst_train_ds, lst_eval_set, tokenizer, data_collator), (lst_train_ds_genr, lst_eval_set_genr) 


In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()